In [3]:
pip install pydrive

Processing /Users/lucas_botella/Library/Caches/pip/wheels/c6/14/12/ccdcc5d3b41661f360f9c7d9f7ea9d1879a5f85aa4ecc8cc6f/PyDrive-1.3.1-py3-none-any.whl
  Using cached oauth2client-4.1.3-py2.py3-none-any.whl (98 kB)
Note: you may need to restart the kernel to use updated packages.


In [91]:
# Import the Libraries
from pydrive.auth import GoogleAuth
import pandas as pd
from pydrive.drive import GoogleDrive
import glob

In [92]:
# A browser window will open. login using the appropriate account.
gauth = GoogleAuth()
gauth.LocalWebserverAuth() #

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=267819126046-3da5rmgml4iemor49ermcv23tnaj02t5.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [93]:
drive = GoogleDrive(gauth)

In [94]:
drive

## Retrieve data from Google Drive

In [ ]:
#reads a csv file and return a dataframe, if not found returns empty dataframe
def read_csv_file(folder, fileName):
    
    listed_mortality = drive.ListFile({'q': "title contains '" + fileName + ".csv' and '" + folder + "' in parents"}).GetList()
    
    if len(listed_mortality)>0:
        title = listed_mortality[0]['title']
        id = listed_mortality[0]['id']
        each_file = drive.CreateFile({'id': id})
        each_file.GetContentFile(title)
        df = pd.read_csv(title)
    else:
        
        df=pd.DataFrame()
    
    return df

In [95]:
def group_policies(filename, isMobilityTrend):
    
    if(isMobilityTrend):
        
        df = pd.read_csv(filename)
        df['Date'] = pd.to_datetime(df['Date']) + pd.to_timedelta(11, unit='d')
        df = df.groupby(['Code', pd.Grouper(key='Date', freq='W-THU')]).mean().reset_index()
        df = df.groupby(['Code', pd.Grouper(key='Date', freq='W-MON', label='left',closed='left')]).mean().reset_index()
        
    else:
        
        df = pd.read_csv(filename)
        df['Date'] = pd.to_datetime(df['Date']) + pd.to_timedelta(18, unit='d')
        df = df.groupby(['Code', pd.Grouper(key='Date', freq='W-THU')]).mean().reset_index()
        df = df.groupby(['Code', pd.Grouper(key='Date', freq='W-MON', label='left',closed='left')]).mean().reset_index()
        
    
    return df

In [96]:
def df_group_policies_trends_others():
    listed = drive.ListFile({'q': "title contains '.csv' and '1noRbLRgnQ5vx3PiQy1RxReIeYQjKuZBJ' in parents"}).GetList()
    list_id = []
    list_title = []
    for file in listed:
        list_id.append(file['id'])
        list_title.append(file['title'])

    df = pd.DataFrame()

    i=0

    for id, title in zip(list_id, list_title):
        print(title)
        each_file = drive.CreateFile({'id': id})
        each_file.GetContentFile(title)

        if(i==0):


            if(title == 'changes-visitors-covid.csv'):

                df = group_policies(title,True)

            else:    

                df = group_policies(title,False)

            i=i+1

        else:

            if(title == 'changes-visitors-covid.csv'):

                df_aux = group_policies(title,True)

            else:    

                df_aux = group_policies(title,False)

            i=i+1


            df = pd.merge(df, df_aux, on=["Code", "Date"])
            
    return df


# Create Dataframe with all Features

## Policies and Mobility Trends

In [141]:
df_policies=df_group_policies_trends_others()

workplace-visitors-covid.csv
visitors-transit-covid.csv
changes-visitors-covid.csv
debt-relief-covid.csv
income-support-covid.csv
covid-contact-tracing.csv
covid-19-testing-policy.csv
international-travel-covid.csv
internal-movement-covid.csv
public-transport-covid.csv
public-campaigns-covid.csv
face-covering-policies-covid.csv
stay-at-home-covid.csv
public-gathering-rules-covid.csv
public-events-covid.csv
workplace-closures-covid.csv
school-closures-covid.csv


In [203]:
df_policies

,Code,Date,workplaces_x,transit_stations_x,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations_y,workplaces_y,residential,...,international_travel_controls,restrictions_internal_movements,close_public_transport,public_information_campaigns,facial_coverings,stay_home_requirements,restriction_gatherings,cancel_public_events,workplace_closures,school_closures
0,ABW,2020-03-09,-10.391143,7.585429,0.387714,-3.408000,8.857143,5.224571,-12.734571,2.265286,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,ABW,2020-03-16,-12.734571,5.224571,3.408286,4.816429,7.143000,4.714286,2.367286,-0.020429,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
2,ABW,2020-03-23,2.367286,4.714286,-2.775571,4.694000,6.040857,-0.122571,1.040714,1.204143,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
3,ABW,2020-03-30,1.040714,-0.122571,-34.142714,-3.918286,-15.693857,-24.836857,-23.000000,12.908429,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
4,ABW,2020-04-06,-23.000000,-24.836857,-57.469429,-19.673286,-48.592000,-56.041000,-48.857000,24.214143,...,3.428571,0.0,0.0,1.428571,0.000000,0.285714,0.000000,0.0,0.571429,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6151,ZWE,2021-01-04,7.448857,20.183571,18.530714,37.693857,44.204000,23.122286,-15.428429,5.285571,...,2.000000,2.0,1.0,2.000000,3.000000,2.000000,3.000000,2.0,1.428571,1.000000
6152,ZWE,2021-01-11,-15.428429,23.122286,2.632571,22.959143,35.101857,1.816286,-33.714143,11.755000,...,2.000000,2.0,1.0,2.000000,3.000000,2.000000,3.000000,2.0,2.000000,1.000000
6153,ZWE,2021-01-18,-33.714143,1.816286,-15.653143,6.040857,15.551000,-14.143000,-26.979714,17.918571,...,2.000000,2.0,1.0,2.000000,3.000000,2.000000,3.000000,2.0,2.000000,1.000000
6154,ZWE,2021-01-25,-26.979714,-14.143000,-38.632714,-17.347000,-11.979571,-46.510286,-35.203857,21.408143,...,3.714286,2.0,1.0,2.000000,3.857143,2.000000,3.857143,2.0,2.857143,1.857143


## Excess Mortality 

In [204]:
df_excess_mortality=read_csv_file('12p5R49Bg02b0N__aETBxNThCVnlSDaHv','excess-mortality-p-scores')
df_excess_mortality['Date'] = pd.to_datetime(df_excess_mortality['Date'])
df_excess_mortality = df_excess_mortality.groupby(['Code', pd.Grouper(key='Date', freq='W-MON')]).mean().reset_index()
df_excess_mortality

,Code,Date,"Excess mortality P-scores, all ages"
0,AUT,2020-01-06,-10.78
1,AUT,2020-01-13,-8.35
2,AUT,2020-01-20,0.59
3,AUT,2020-01-27,1.83
4,AUT,2020-02-03,7.06
...,...,...,...
1767,USA,2020-11-23,26.16
1768,USA,2020-11-30,27.51
1769,USA,2020-12-07,31.28
1770,USA,2020-12-14,36.63


In [205]:
df_merge = pd.merge(df_policies, df_excess_mortality, on=["Code", "Date"])
df_merge

,Code,Date,workplaces_x,transit_stations_x,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations_y,workplaces_y,residential,...,restrictions_internal_movements,close_public_transport,public_information_campaigns,facial_coverings,stay_home_requirements,restriction_gatherings,cancel_public_events,workplace_closures,school_closures,"Excess mortality P-scores, all ages"
0,AUT,2020-03-09,-2.385286,5.645000,4.734571,8.775429,21.285714,2.979429,-0.040714,0.020429,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,-0.55
1,AUT,2020-03-16,-0.040714,2.979429,1.163143,5.632571,7.734714,-0.347143,1.530571,0.122571,...,0.000000,0.000000,2.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.39
2,AUT,2020-03-23,1.530571,-0.347143,-3.632714,9.796000,6.979286,-5.755143,-0.796000,0.836714,...,0.000000,0.000000,2.0,0.0,0.0,0.0,0.000000,0.000000,0.0,14.89
3,AUT,2020-03-30,-0.796000,-5.755143,-58.143000,-14.571429,-24.959286,-48.265143,-34.428571,15.224429,...,0.857143,0.428571,2.0,0.0,0.0,0.0,1.428571,0.571429,0.0,7.24
4,AUT,2020-04-06,-34.428571,-48.265143,-82.367286,-36.877571,-49.612143,-71.204000,-58.244857,23.347000,...,2.000000,1.000000,2.0,0.0,2.0,4.0,2.000000,3.000000,2.0,15.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1417,USA,2020-11-23,-25.510286,-32.040571,-17.612143,-8.143000,1.877429,-33.653000,-26.102000,8.591857,...,2.000000,1.000000,2.0,4.0,1.0,3.0,2.000000,2.000000,3.0,26.16
1418,USA,2020-11-30,-26.102000,-33.653000,-19.163286,-7.469429,-7.979714,-34.571429,-26.122286,9.163429,...,2.000000,1.000000,2.0,4.0,1.0,3.0,2.000000,2.000000,3.0,27.51
1419,USA,2020-12-07,-26.122286,-34.571429,-21.367286,-4.510143,-7.489714,-35.775571,-33.877571,10.755286,...,2.000000,1.000000,2.0,4.0,2.0,4.0,2.000000,3.000000,3.0,31.28
1420,USA,2020-12-14,-33.877571,-35.775571,-23.306000,-10.306000,-13.122429,-37.367286,-33.285714,11.203857,...,2.000000,1.000000,2.0,4.0,2.0,4.0,2.000000,3.000000,3.0,36.63


In [148]:
df_merge.Code.unique()

array(['AUT', 'BEL', 'BGR', 'CAN', 'CHE', 'CHL', 'CZE', 'DEU', 'DNK',
       'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GRC', 'HRV', 'HUN', 'ISR',
       'ITA', 'KOR', 'LTU', 'LUX', 'LVA', 'NLD', 'NOR', 'NZL', 'POL',
       'PRT', 'SVK', 'SVN', 'SWE', 'TWN', 'USA'], dtype=object)

## Weather Data

In [150]:
import pycountry

In [195]:
df_weather=read_csv_file('1yXs7YOhttKS7STixDmqD1kv1woRYWfUd','weather_final_df')

In [196]:
df_weather

,Unnamed: 0,country,date,temp,prcp
0,0,US,2020-01-07,6.833333,0.00
1,1,US,2020-01-28,-41.777778,0.00
2,2,US,2020-02-06,-43.777778,0.00
3,3,CA,2020-11-02,8.777778,0.78
4,4,CA,2020-06-17,27.611111,0.00
...,...,...,...,...,...
5038093,5038093,AU,2021-01-14,31.555556,0.08
5038094,5038094,AU,2021-01-26,28.722222,0.20
5038095,5038095,AU,2021-03-12,29.444444,0.01
5038096,5038096,AU,2021-01-06,25.500000,4.41


In [197]:
df_weather=df_weather.dropna()
df_weather

,Unnamed: 0,country,date,temp,prcp
0,0,US,2020-01-07,6.833333,0.00
1,1,US,2020-01-28,-41.777778,0.00
2,2,US,2020-02-06,-43.777778,0.00
3,3,CA,2020-11-02,8.777778,0.78
4,4,CA,2020-06-17,27.611111,0.00
...,...,...,...,...,...
5038093,5038093,AU,2021-01-14,31.555556,0.08
5038094,5038094,AU,2021-01-26,28.722222,0.20
5038095,5038095,AU,2021-03-12,29.444444,0.01
5038096,5038096,AU,2021-01-06,25.500000,4.41


In [198]:
for country in pycountry.countries: 
    df_weather.loc[df_weather['country']==country.alpha_2, 'Code'] = country.alpha_3

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [199]:
df_weather=df_weather[['Code','date', 'temp', 'prcp']]
df_weather['Date'] = pd.to_datetime(df_weather['date']) + pd.to_timedelta(18, unit='d')
df_weather = df_weather.groupby(['Code', pd.Grouper(key='Date', freq='W-MON')]).mean().reset_index()
df_weather

<ipython-input-199-adb7c95f69e7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weather['Date'] = pd.to_datetime(df_weather['date']) + pd.to_timedelta(18, unit='d')


,Code,Date,temp,prcp
0,AFG,2020-01-20,3.435484,0.105161
1,AFG,2020-01-27,1.952452,0.284324
2,AFG,2020-02-03,-1.335430,0.100943
3,AFG,2020-02-10,-0.162963,0.082857
4,AFG,2020-02-17,1.824159,0.030734
...,...,...,...,...
11956,ZWE,2021-03-22,23.727273,0.097273
11957,ZWE,2021-03-29,24.879928,0.000323
11958,ZWE,2021-04-05,24.672619,0.002857
11959,ZWE,2021-04-12,23.386667,0.000400


In [206]:
df_merge = pd.merge(df_merge, df_weather, on=["Code", "Date"])
df_merge

,Code,Date,workplaces_x,transit_stations_x,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations_y,workplaces_y,residential,...,public_information_campaigns,facial_coverings,stay_home_requirements,restriction_gatherings,cancel_public_events,workplace_closures,school_closures,"Excess mortality P-scores, all ages",temp,prcp
0,AUT,2020-03-09,-2.385286,5.645000,4.734571,8.775429,21.285714,2.979429,-0.040714,0.020429,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,-0.55,4.361230,0.027141
1,AUT,2020-03-16,-0.040714,2.979429,1.163143,5.632571,7.734714,-0.347143,1.530571,0.122571,...,2.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.39,4.831166,0.061951
2,AUT,2020-03-23,1.530571,-0.347143,-3.632714,9.796000,6.979286,-5.755143,-0.796000,0.836714,...,2.0,0.0,0.0,0.0,0.000000,0.000000,0.0,14.89,3.537560,0.086404
3,AUT,2020-03-30,-0.796000,-5.755143,-58.143000,-14.571429,-24.959286,-48.265143,-34.428571,15.224429,...,2.0,0.0,0.0,0.0,1.428571,0.571429,0.0,7.24,5.728351,0.079613
4,AUT,2020-04-06,-34.428571,-48.265143,-82.367286,-36.877571,-49.612143,-71.204000,-58.244857,23.347000,...,2.0,0.0,2.0,4.0,2.000000,3.000000,2.0,15.30,6.210833,0.011932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1416,USA,2020-11-23,-25.510286,-32.040571,-17.612143,-8.143000,1.877429,-33.653000,-26.102000,8.591857,...,2.0,4.0,1.0,3.0,2.000000,2.000000,3.0,26.16,9.105515,0.040582
1417,USA,2020-11-30,-26.102000,-33.653000,-19.163286,-7.469429,-7.979714,-34.571429,-26.122286,9.163429,...,2.0,4.0,1.0,3.0,2.000000,2.000000,3.0,27.51,11.292633,0.071417
1418,USA,2020-12-07,-26.122286,-34.571429,-21.367286,-4.510143,-7.489714,-35.775571,-33.877571,10.755286,...,2.0,4.0,2.0,4.0,2.000000,3.000000,3.0,31.28,7.238694,0.050378
1419,USA,2020-12-14,-33.877571,-35.775571,-23.306000,-10.306000,-13.122429,-37.367286,-33.285714,11.203857,...,2.0,4.0,2.0,4.0,2.000000,3.000000,3.0,36.63,7.033025,0.060010


## Doctors and Nurses

In [208]:
doctors_df=read_csv_file('17wrWTb6BZMqWkNrqYipTQXo7IOxqekOf','doctors')
nurses_df=read_csv_file('17wrWTb6BZMqWkNrqYipTQXo7IOxqekOf','nurses')

In [218]:
doctors_df=doctors_df[['LOCATION', 'TIME', 'Value']]
doctors_df_18=doctors_df[doctors_df['TIME'] == 2018]
nurses_df=nurses_df[['LOCATION', 'TIME', 'Value']]
nurses_df=nurses_df[nurses_df['TIME'] == 2018]

In [226]:
for index, row in doctors_df.iterrows(): 
    df_merge.loc[df_merge['Code']==row.LOCATION , 'doctors_per_1000'] = row.Value
    
for index, row in nurses_df.iterrows(): 
    df_merge.loc[df_merge['Code']==row.LOCATION , 'nurses_per_1000'] = row.Value

In [227]:
df_merge

,Code,Date,workplaces_x,transit_stations_x,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations_y,workplaces_y,residential,...,stay_home_requirements,restriction_gatherings,cancel_public_events,workplace_closures,school_closures,"Excess mortality P-scores, all ages",temp,prcp,doctors_per_1000,nurses_per_1000
0,AUT,2020-03-09,-2.385286,5.645000,4.734571,8.775429,21.285714,2.979429,-0.040714,0.020429,...,0.0,0.0,0.000000,0.000000,0.0,-0.55,4.361230,0.027141,5.24,6.87
1,AUT,2020-03-16,-0.040714,2.979429,1.163143,5.632571,7.734714,-0.347143,1.530571,0.122571,...,0.0,0.0,0.000000,0.000000,0.0,0.39,4.831166,0.061951,5.24,6.87
2,AUT,2020-03-23,1.530571,-0.347143,-3.632714,9.796000,6.979286,-5.755143,-0.796000,0.836714,...,0.0,0.0,0.000000,0.000000,0.0,14.89,3.537560,0.086404,5.24,6.87
3,AUT,2020-03-30,-0.796000,-5.755143,-58.143000,-14.571429,-24.959286,-48.265143,-34.428571,15.224429,...,0.0,0.0,1.428571,0.571429,0.0,7.24,5.728351,0.079613,5.24,6.87
4,AUT,2020-04-06,-34.428571,-48.265143,-82.367286,-36.877571,-49.612143,-71.204000,-58.244857,23.347000,...,2.0,4.0,2.000000,3.000000,2.0,15.30,6.210833,0.011932,5.24,6.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1416,USA,2020-11-23,-25.510286,-32.040571,-17.612143,-8.143000,1.877429,-33.653000,-26.102000,8.591857,...,1.0,3.0,2.000000,2.000000,3.0,26.16,9.105515,0.040582,2.61,11.89
1417,USA,2020-11-30,-26.102000,-33.653000,-19.163286,-7.469429,-7.979714,-34.571429,-26.122286,9.163429,...,1.0,3.0,2.000000,2.000000,3.0,27.51,11.292633,0.071417,2.61,11.89
1418,USA,2020-12-07,-26.122286,-34.571429,-21.367286,-4.510143,-7.489714,-35.775571,-33.877571,10.755286,...,2.0,4.0,2.000000,3.000000,3.0,31.28,7.238694,0.050378,2.61,11.89
1419,USA,2020-12-14,-33.877571,-35.775571,-23.306000,-10.306000,-13.122429,-37.367286,-33.285714,11.203857,...,2.0,4.0,2.000000,3.000000,3.0,36.63,7.033025,0.060010,2.61,11.89


## Hospital Beds

In [230]:
hospital_beds_df=read_csv_file('1YlfutCzBqYwPhZw-7OsfLC3k1uv1LYfa','hospital_beds')
hospital_beds_df

TypeError: read_csv_file() got an unexpected keyword argument 'skiprows'

## Correlation Matrix

In [228]:
rs = np.random.RandomState(0)
df = pd.DataFrame(rs.rand(10, 10))
corr = df_merge.corr()
corr.style.background_gradient(cmap='coolwarm')

,workplaces_x,transit_stations_x,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations_y,workplaces_y,residential,debt_relief,income_support,contact_tracing,testing_policy,international_travel_controls,restrictions_internal_movements,close_public_transport,public_information_campaigns,facial_coverings,stay_home_requirements,restriction_gatherings,cancel_public_events,workplace_closures,school_closures,"Excess mortality P-scores, all ages",temp,prcp,doctors_per_1000,nurses_per_1000
workplaces_x,1.000000,0.822092,0.693568,0.661681,0.119936,0.729918,0.879135,-0.726217,-0.292454,-0.371721,0.106945,0.033506,-0.263685,-0.510854,-0.414866,-0.236895,-0.028324,-0.541753,-0.570065,-0.517024,-0.660077,-0.620163,-0.166405,0.129291,0.163579,-0.055642,-0.003692
transit_stations_x,0.822092,1.000000,0.793561,0.742831,0.409055,0.930247,0.756523,-0.837534,-0.245451,-0.365251,0.096757,0.037277,-0.390669,-0.570216,-0.431125,-0.263859,0.008876,-0.608552,-0.585011,-0.579148,-0.655349,-0.606673,-0.196743,0.274104,0.128930,-0.083642,-0.048554
retail_and_recreation,0.693568,0.793561,1.000000,0.808885,0.629376,0.868998,0.770334,-0.929342,-0.140380,-0.131348,0.082023,0.164478,-0.184821,-0.552504,-0.374518,-0.141218,0.015780,-0.599116,-0.499614,-0.558519,-0.608325,-0.579204,-0.364973,0.345520,0.142782,-0.048197,0.065761
grocery_and_pharmacy,0.661681,0.742831,0.808885,1.000000,0.549346,0.792526,0.707375,-0.834371,-0.189715,-0.116093,0.054453,0.154106,-0.187043,-0.428742,-0.360265,-0.211978,0.061322,-0.476794,-0.369939,-0.459624,-0.504854,-0.488659,-0.240154,0.249124,0.078923,-0.010999,0.016153
parks,0.119936,0.409055,0.629376,0.549346,1.000000,0.486382,0.168662,-0.592005,0.032516,0.173425,0.037437,0.168601,-0.030653,-0.282183,-0.128102,0.023655,-0.035769,-0.336978,-0.167949,-0.262373,-0.203040,-0.175268,-0.280875,0.380854,0.003843,-0.010747,0.068144
transit_stations_y,0.729918,0.930247,0.868998,0.792526,0.486382,1.000000,0.814913,-0.915954,-0.162949,-0.272295,0.134812,0.098084,-0.335495,-0.538900,-0.398608,-0.260194,0.072231,-0.576357,-0.538091,-0.570394,-0.616565,-0.584009,-0.287109,0.342371,0.146147,-0.087240,-0.047650
workplaces_y,0.879135,0.756523,0.770334,0.707375,0.168662,0.814913,1.000000,-0.824247,-0.221631,-0.296746,0.146819,0.087485,-0.224301,-0.485160,-0.393311,-0.253054,0.039459,-0.517121,-0.535764,-0.519037,-0.631522,-0.612071,-0.263716,0.192257,0.163636,-0.055577,0.000642
residential,-0.726217,-0.837534,-0.929342,-0.834371,-0.592005,-0.915954,-0.824247,1.000000,0.119246,0.150412,-0.137339,-0.137817,0.211300,0.569401,0.424290,0.189028,-0.076472,0.649671,0.508100,0.549605,0.629598,0.588135,0.336732,-0.344372,-0.140811,0.037627,0.007129
debt_relief,-0.292454,-0.245451,-0.140380,-0.189715,0.032516,-0.162949,-0.221631,0.119246,1.000000,0.258929,0.142696,0.248733,0.184930,0.080896,0.246454,0.234731,0.339338,0.296487,0.242269,0.156816,0.266254,0.277305,0.012805,0.281311,-0.054362,-0.065214,-0.581220
income_support,-0.371721,-0.365251,-0.131348,-0.116093,0.173425,-0.272295,-0.296746,0.150412,0.258929,1.000000,0.209977,0.242471,0.361249,0.303272,0.182812,0.314181,0.257333,0.248174,0.452946,0.345126,0.337638,0.176687,0.099238,0.121989,-0.030960,0.004922,0.242905
